In [1]:
# Autenticar tu cuenta de Google
from google.colab import auth
auth.authenticate_user()
import pandas as pd
import io
# Configurar las credenciales
from google.cloud import storage
client = storage.Client()

# Specify the bucket and file names
bucket_name = 'etl_archivos'
file_name = 'ReviewsYelp.parquet'

# Create a storage client
storage_client = storage.Client()

# Get the bucket
bucket = storage_client.get_bucket(bucket_name)

# Get the blob (file)
blob = bucket.get_blob(file_name)

# Download the blob's contents to a BytesIO object
parquet_bytes = io.BytesIO(blob.download_as_string())

# Read the parquet file into a DataFrame using pandas
df_join_yelp = pd.read_parquet(parquet_bytes)


In [2]:
# Specify the bucket and file names
bucket_name = 'etl_archivos'
file_name = 'ReviewsUnificados.parquet'

# Create a storage client
storage_client = storage.Client()

# Get the bucket
bucket = storage_client.get_bucket(bucket_name)

# Get the blob (file)
blob = bucket.get_blob(file_name)

# Download the blob's contents to a BytesIO object
parquet_bytes = io.BytesIO(blob.download_as_string())

# Read the parquet file into a DataFrame using pandas
df_join_yelp2 = pd.read_parquet(parquet_bytes)
df_join_yelp2.drop(columns=['review_id'], inplace = True)
df_join_yelp_concat = pd.concat([df_join_yelp, df_join_yelp2], ignore_index=True)

In [3]:
import uuid
df_join_yelp_concat['review_id'] = [uuid.uuid4() for _ in range(len(df_join_yelp_concat))]

In [4]:
# Create a StringIO object to hold the DataFrame in memory
sio = io.BytesIO()
df_join_yelp_concat['rating'] = df_join_yelp_concat['rating'].astype(str)
df_join_yelp_concat['review_id'] = df_join_yelp_concat['review_id'].astype(str)
# Write the DataFrame to the StringIO object in Parquet format
df_join_yelp_concat.to_parquet(sio)

# Create a client object for interacting with Cloud Storage
client = storage.Client()

# Create a bucket object for the desired bucket
bucket = client.get_bucket('etl_archivos')

# Create a blob object for the desired file
blob = bucket.blob('ReviewsTodasFinal.parquet')

# Upload the DataFrame from the StringIO object to the blob
blob.upload_from_string(sio.getvalue())